# Import

In [1]:
import pandas as pd
import numpy as np
import random

# Function

In [5]:
### Get points
def get_points(data='att48.txt'):
    with open(data) as f:
        line = f.readlines()
        list_data = []
        for index, data in enumerate(line):
            list_data.append(data.strip())

    points = []
    for i in range(len(list_data)):
        points.append((int(list_data[i].split(' ')[1]), int(list_data[i].split(' ')[2])))
    return points

### Get distance
def distance(point_i : int, point_j: int):
    return ((point_i[0] - point_j[0])**2 + (point_i[1] - point_j[1])**2)**(1/2)

def get_cost_matrix():
    cost_matrix = []
    for i in range(all_points):
        row = []
        for j in range(all_points):
            row.append(distance(get_points()[i], get_points()[j]))
        cost_matrix.append(row)
    return np.array(cost_matrix)


### DataFrame
def df(list_gen, list_ant, list_distance, list_path):
    df = pd.DataFrame({'Gen' : list_gen, 'Ant' : list_ant,  'Distance' : list_distance, 'Path' : list_path})
    # min_distance = df[df.Distance == df.Distance.min()].Path.values[0]
    # for i in range(len(min_distance)):
    #     if i != len(paths)-1:
    #         t_new = ((1-p)*t0[min_distance[i]][min_distance[i+1]]) + (1/total_distance)
    #         t0[paths[i]][paths[i+1]] = t_new
    return df

### Reset values
def reset_values():
    get_cost_matrix()

In [7]:
### Default values
all_points = len(get_points())
cost_matrix = get_cost_matrix() # Distance between i & j
cm = cost_matrix.copy()
n = np.array([[0 if i == j else 1 / cost_matrix[i][j] for i in range(all_points)] for j in range(all_points)]) # 1/distance
gens= 50
ants= 48
alpha= 3
beta= 3
p= .02

t0 = np.array([[0 if i==j else 1 for i in range(all_points)] for j in range(all_points)], dtype=float) # Default pheromone

list_gens = []
list_all_paths = []
list_all_total_distance = []
list_shortest_distance_value = []

list_pheromones = {}
list_shortest_distance_value = []
list_shortest_distance_index = []
list_shortest_path = []
for gen in range(gens):
    print(f'Gen : {gen}')

    list_ants = []
    list_paths = {}
    list_total_distance = {}

    random_start = list(range(all_points))
    for ant in range(ants):
        print(f'Ant : {ant}')

        allowed = [i for i in np.arange(all_points)]
        paths = []

        ### -------------------------------------------------- Start --------------------------------------------------
        start = random_start.pop(random.randrange(len(random_start))) # Random between 0 to 47
        selected = start

        paths.append(selected)

        # print(f'Start : {selected}')
        best_path = np.where(cost_matrix[paths[-1]] == sorted(cost_matrix[paths[-1]])[1])[0][0] # Return array, # sorted(cost_matrix[paths[-1]])[1] : ค่าที่ใกล้สุด
        distance_ij = sorted(cost_matrix[paths[-1]])[1] # index[0] : 0
        # print(f'Best paths : {best_path} ,Distance : {distance_ij}')
        # print(paths)
        ### Recheck best path
        # for best in enumerate(cost_matrix[paths[-1]]):
        #     print(best)

        allowed.remove(selected)
        for i in range(all_points):
            cost_matrix[selected][i] = 0
            cost_matrix[i][selected] = 0

        ### -------------------------------------------------- Denominator --------------------------------------------------
        list_t0 = []
        list_n = []
        for i in allowed:
            # print(f'Proromone [{selected}][{i}] : {t0[selected][i]} ** {alpha} = {t0[selected][i] ** alpha}')
            # print(f'n [{selected}][{i}] : {n[selected][i]} ** {beta} = {n[selected][i] ** beta}')
            list_t0.append(t0[selected][i] ** alpha)
            list_n.append(n[selected][i] ** beta)
        list_t0 = np.array(list_t0)
        list_n = np.array(list_n)
        denominator = sum(list_t0 * list_n)
        # print(f'Denominator : {denominator}')
        # print('-'*50)
        
        ### -------------------------------------------------- Probability --------------------------------------------------
        for city in range(len(allowed)):
            prob = {}
            for i in allowed:
                prob.update({i: (t0[paths[-1]][i]**alpha) * (n[paths[-1]][i]**beta) / denominator})
            max_prob = sorted(prob.values())[-1]
            selected = list(prob.keys())[list(prob.values()).index(max_prob)]

            # for key, value in zip(prob.keys(), prob.values()):
            #     print(key, value)

            paths.append(selected)

            # print(f'Allowed : {allowed}')
            # print(f'Select : {selected}')

            ### Check walked path
            walked = 0
            for i in cost_matrix[paths[-1]]:
                if i == 0:
                    walked += 1
            # print(f'walked : {walked}')
            if walked != all_points:
                best_path = np.where(cost_matrix[paths[-1]] == sorted(cost_matrix[paths[-1]])[walked])[0][0]
                distance_ij = sorted(cost_matrix[paths[-1]])[walked]
                # print(f'Best paths : {best_path} ,Distance : {distance_ij}')

            # print(f'Path : {paths}')
            ### Recheck best path
            # for best in enumerate(cost_matrix[paths[-1]]):
            #     print(best)

            ### Remove the path that has been walked
            allowed.remove(selected)
            for i in range(all_points):
                cost_matrix[selected][i] = 0
                cost_matrix[i][selected] = 0

            ### -------------------------------------------------- Denominator --------------------------------------------------
            list_t0 = []
            list_n = []
            for i in allowed:
                # print(f'Proromone [{selected}][{i}] : {t0[selected][i]} ** {alpha} = {t0[selected][i] ** alpha}')
                # print(f'n [{selected}][{i}] : {n[selected][i]} ** {beta} = {n[selected][i] ** beta}')
                list_t0.append(t0[selected][i] ** alpha)
                list_n.append(n[selected][i] ** beta)
            list_t0 = np.array(list_t0)
            list_n = np.array(list_n)
            denominator = sum(list_t0 * list_n)
            # print(f'Denominator : {denominator}')
            # print('-'*50)

        ### -------------------------------------------------- Total distance --------------------------------------------------
        total_distance = 0
        for i in range(len(paths)):
            if i != len(paths)-1:
                total_distance += (cm[paths[i]][paths[i+1]])
        # print(f'Total Distance : {total_distance}')

        list_ants.append(ant)
        list_total_distance.update({ant : total_distance})
        list_paths.update({ant : paths})
        # print(paths)


#         print(f'list_gens : {list_gens}')
#         print(f'list_total_distance : {list_total_distance}')
    
        # print(f'List total distance : {list_total_distance}')
        # print(f'List total path : {list_paths}')

        ### -------------------------------------------------- Reset cost matrix --------------------------------------------------
        reset_values()

    ### -------------------------------------------------- Update pheromone --------------------------------------------------
    ### Pair paths
    list_pair_paths = []
    for i in range(len(list_paths)):
        pair_paths = []
        for j in range(len(list_paths)):
            try:
                pair_paths.append([list_paths[i][j], list_paths[i][j+1]])
            except:
                pass
        list_pair_paths.append(pair_paths)
    
    ### Count paths for update pheromone
    count = {}
    for i, _distance in zip(range(len(list_pair_paths)), list(list_total_distance.values())):
        for j in list_pair_paths[i]:
            if not str(j) in count:
                count[str(j)] = 1
            else:
                count[str(j)] += 1

    list_count = []
    for i in list(count.keys()):
        i = i.split(',')
        count_0 = int(i[0].replace('[', ''))
        count_1 = int(i[1].replace(']', ''))
        list_count.append([count_0, count_1])

    for i in list_count:
        t0[i[0]][i[1]] += (1-p)*t0[i[0]][i[1]]
        # t0[i[1]][i[0]] += (1-p)*t0[i[1]][i[0]]

    ### Plus by summation walked path
    for i, _distance in zip(range(len(list_pair_paths)), list(list_total_distance.values())):
        for j in list_pair_paths[i]:
            t0[j[0]][j[1]] += 100/_distance
            # t0[j[1]][j[0]] += 1/_distance

            # print(f"t0[j[0]][j[1]] : {t0[j[0]][j[1]]}")
            # print(f"t0[j[1]][j[0]] : {t0[j[1]][j[0]]}")
            # print(j[0], j[1],distance)
            # print(j[1], j[0],distance)
    #         # print("------------------")

#     print(list_paths)
    shortest_distance_value = sorted(list_total_distance.values())[0]
    shortest_distance_index = list(list_total_distance.keys())[list(list_total_distance.values()).index(shortest_distance_value)]
    shortest_path = list_paths[shortest_distance_index]
    # print(f'Shortest distance value: {shortest_distance_value}')
    # print(f'Shortest distance index: {shortest_distance_index}')
    # print(f'Shortest path: {shortest_path}')
    list_shortest_distance_value.append(shortest_distance_value)
    list_shortest_distance_index.append(shortest_distance_index)
    list_shortest_path.append(shortest_path)
#     print(f'-------------')

    list_gens.append(list_ants)
    list_all_paths.append(list_paths)
    list_all_total_distance.append(list_total_distance)

    # for i in range(len(list_all_paths)):
    #     print(list_all_paths[i].values())

    ### Pheromone reset
    # reset_values()
print(f'List of shortest distance values : {sorted(list_shortest_distance_value)}')
# print(list_pheromones)
# return [list_gens, list_ants, list_total_distance, list_paths]
    

Gen : 0
Ant : 0
Ant : 1
Ant : 2
Ant : 3
Ant : 4
Ant : 5
Ant : 6
Ant : 7
Ant : 8
Ant : 9
Ant : 10
Ant : 11
Ant : 12
Ant : 13
Ant : 14
Ant : 15
Ant : 16
Ant : 17
Ant : 18
Ant : 19
Ant : 20
Ant : 21
Ant : 22
Ant : 23
Ant : 24
Ant : 25
Ant : 26
Ant : 27
Ant : 28
Ant : 29
Ant : 30
Ant : 31
Ant : 32
Ant : 33
Ant : 34
Ant : 35
Ant : 36
Ant : 37
Ant : 38
Ant : 39
Ant : 40
Ant : 41
Ant : 42
Ant : 43
Ant : 44
Ant : 45
Ant : 46
Ant : 47
Gen : 1
Ant : 0
Ant : 1
Ant : 2
Ant : 3
Ant : 4
Ant : 5
Ant : 6
Ant : 7
Ant : 8
Ant : 9
Ant : 10
Ant : 11
Ant : 12
Ant : 13
Ant : 14
Ant : 15
Ant : 16
Ant : 17
Ant : 18
Ant : 19
Ant : 20
Ant : 21
Ant : 22
Ant : 23
Ant : 24
Ant : 25
Ant : 26
Ant : 27
Ant : 28
Ant : 29
Ant : 30
Ant : 31
Ant : 32
Ant : 33
Ant : 34
Ant : 35
Ant : 36
Ant : 37
Ant : 38
Ant : 39
Ant : 40
Ant : 41
Ant : 42
Ant : 43
Ant : 44
Ant : 45
Ant : 46
Ant : 47
Gen : 2
Ant : 0
Ant : 1
Ant : 2
Ant : 3
Ant : 4
Ant : 5
Ant : 6
Ant : 7
Ant : 8
Ant : 9
Ant : 10
Ant : 11
Ant : 12
Ant : 13
Ant : 14
Ant : 1

KeyboardInterrupt: 

In [8]:
list_shortest_distance_value

[35135.455097450664,
 35135.455097450664,
 35135.455097450664,
 35135.455097450664,
 35135.455097450664,
 35135.455097450664,
 35135.455097450664,
 35135.455097450664,
 35135.455097450664,
 35135.455097450664,
 35135.455097450664,
 35135.455097450664,
 35135.455097450664,
 35135.455097450664,
 35135.455097450664,
 35135.455097450664,
 35135.455097450664,
 35135.455097450664,
 35135.455097450664,
 35135.455097450664]